In [ ]:
import pandas as pd

In [1]:
# Charger les fichiers CSV
operations = pd.read_csv("data/operations.csv")
resultats_humain = pd.read_csv("data/resultats_humain.csv")
flotteurs = pd.read_csv("data/flotteurs.csv")

# Afficher les premières lignes
print("=== OPERATIONS ===")
print(operations.head(), "\n")

print("=== RESULTATS HUMAIN ===")
print(resultats_humain.head(), "\n")

print("=== FLOTTEURS ===")
print(flotteurs.head(), "\n")

# Vérifier les infos sur les colonnes et types
print("=== INFO OPERATIONS ===")
print(operations.info(), "\n")

print("=== INFO RESULTATS HUMAIN ===")
print(resultats_humain.info(), "\n")

print("=== INFO FLOTTEURS ===")
print(flotteurs.info(), "\n")


NameError: name 'pd' is not defined

In [ ]:


def audit_table(df, table_name):
    print(f"\n================ AUDIT : {table_name} ================")

    # Dimensions
    print(f"Nombre de lignes : {df.shape[0]}")
    print(f"Nombre de colonnes : {df.shape[1]}")

    # Colonnes
    print("\nColonnes :")
    print(df.columns.tolist())

    # Types
    print("\nTypes des colonnes :")
    print(df.dtypes)

    # Valeurs manquantes
    print("\nValeurs manquantes :")
    print(df.isna().sum())

    # Doublons
    print("\nNombre de doublons :")
    print(df.duplicated().sum())

    # Statistiques descriptives
    print("\nStatistiques descriptives :")
    print(df.describe(include='all'))

    # Vérification des colonnes date/heure
    print("\nVérification des colonnes date/heure :")
    for col in df.columns:
        if "date" in col.lower() or "time" in col.lower():
            try:
                pd.to_datetime(df[col])
                print(f"✔ {col} : format date valide")
            except:
                print(f"❌ {col} : format date NON valide")

    print("=====================================================\n")


In [ ]:
tables = {
    "operations": operations,
    "resultats_humain": resultats_humain,
    "flotteurs": flotteurs
}

for name, df in tables.items():
    audit_table(df, name)


In [ ]:
print("=== Vérification des operation_id ===")

if "operation_id" in operations.columns:
    ops_ids = set(operations["operation_id"])
else:
    ops_ids = set()

for name, df in tables.items():
    if "operation_id" in df.columns:
        ids = set(df["operation_id"])
        print(f"\nTable {name} :")
        print(f"- Nombre d'IDs uniques : {len(ids)}")
        print(f"- IDs présents ici mais absents dans operations : {ids - ops_ids}")


In [ ]:
def resume_table(df, table_name):
    return {
        "table": table_name,
        "rows": df.shape[0],
        "cols": df.shape[1],
        "missing_values": df.isna().sum().sum(),
        "duplicates": df.duplicated().sum()
    }

resume_global = [resume_table(df, name) for name, df in tables.items()]
pd.DataFrame(resume_global)


In [ ]:
expected_columns = {
    "operations": ["operation_id", "date", "type", "lieu"],
    "resultats_humain": ["operation_id", "nb_sauves", "nb_deces"],
    "flotteurs": ["operation_id", "flotteur_id", "etat"]
}

for name, df in tables.items():
    print(f"\n=== Vérification colonnes attendues : {name} ===")
    missing = set(expected_columns[name]) - set(df.columns)
    extra = set(df.columns) - set(expected_columns[name])
    print("Colonnes manquantes :", missing)
    print("Colonnes en trop :", extra)


In [ ]:
print("\n=== Vérification avancée des operation_id ===")

if "operation_id" in operations.columns:
    ops_unique = operations["operation_id"].nunique()
    ops_dups = operations["operation_id"].duplicated().sum()
    print(f"IDs uniques dans operations : {ops_unique}")
    print(f"Doublons dans operations : {ops_dups}")

for name, df in tables.items():
    if "operation_id" in df.columns:
        orphan = set(df["operation_id"]) - set(operations["operation_id"])
        print(f"\nTable {name} : {len(orphan)} IDs orphelins")
        if len(orphan) > 0:
            print(orphan)
